## IBM Applied Data Science Capstone

#### Week 5 - Report

###### Opening a new shopping Mall in Germany

- Build a dataframe of districs in Germany by web scraping the data from Wikipedia page
- Get the geographical coordinates of the districts
- Obtain the venue data for the distrits from Foursquare API
- Explore and cluster the districs
- Select the best cluster to open a new shopping mall

##### 1. Import Libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!conda install -c conda-forge geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! conda install -c conda-forge folium
import folium # map rendering library

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    python_abi:    

##### 2. Scrap data from Wikipedia page into a DataFrame

In [48]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/List_of_cities_in_Germany_by_population").text

In [49]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'lxml')

In [50]:
my_table = soup.find('table',{'class':'wikitable sortable'})
# my_table

In [51]:
links = my_table.find_all('a')
# links

In [52]:
# create a list to store district data
districtList = []

for link in links: 
        districtList.append(link.get('title'))
        
print(districtList)

['States of Germany', 'Berlin', 'Berlin', None, 'Hamburg', 'Hamburg', None, 'Munich', 'Bavaria', None, 'Cologne', 'North Rhine-Westphalia', None, 'Frankfurt', 'Hesse', None, 'Stuttgart', 'Baden-Württemberg', None, 'Düsseldorf', 'North Rhine-Westphalia', None, 'Dortmund', 'North Rhine-Westphalia', None, 'Essen', 'North Rhine-Westphalia', None, 'Leipzig', 'Saxony', None, 'Bremen (state)', 'Bremen (state)', None, 'Dresden', 'Saxony', None, 'Hanover', 'Lower Saxony', None, 'Nuremberg', 'Bavaria', None, 'Duisburg', 'North Rhine-Westphalia', None, 'Bochum', 'North Rhine-Westphalia', None, 'Wuppertal', 'North Rhine-Westphalia', None, 'Bielefeld', 'North Rhine-Westphalia', None, 'Bonn', 'North Rhine-Westphalia', None, 'Münster', 'North Rhine-Westphalia', None, 'Karlsruhe', 'Baden-Württemberg', None, 'Mannheim', 'Baden-Württemberg', None, 'Augsburg', 'Bavaria', None, 'Wiesbaden', 'Hesse', None, 'Gelsenkirchen', 'North Rhine-Westphalia', None, 'Mönchengladbach', 'North Rhine-Westphalia', None, '

In [53]:
del districtList[::3]
districtList

['Berlin',
 'Berlin',
 'Hamburg',
 'Hamburg',
 'Munich',
 'Bavaria',
 'Cologne',
 'North Rhine-Westphalia',
 'Frankfurt',
 'Hesse',
 'Stuttgart',
 'Baden-Württemberg',
 'Düsseldorf',
 'North Rhine-Westphalia',
 'Dortmund',
 'North Rhine-Westphalia',
 'Essen',
 'North Rhine-Westphalia',
 'Leipzig',
 'Saxony',
 'Bremen (state)',
 'Bremen (state)',
 'Dresden',
 'Saxony',
 'Hanover',
 'Lower Saxony',
 'Nuremberg',
 'Bavaria',
 'Duisburg',
 'North Rhine-Westphalia',
 'Bochum',
 'North Rhine-Westphalia',
 'Wuppertal',
 'North Rhine-Westphalia',
 'Bielefeld',
 'North Rhine-Westphalia',
 'Bonn',
 'North Rhine-Westphalia',
 'Münster',
 'North Rhine-Westphalia',
 'Karlsruhe',
 'Baden-Württemberg',
 'Mannheim',
 'Baden-Württemberg',
 'Augsburg',
 'Bavaria',
 'Wiesbaden',
 'Hesse',
 'Gelsenkirchen',
 'North Rhine-Westphalia',
 'Mönchengladbach',
 'North Rhine-Westphalia',
 'Braunschweig',
 'Lower Saxony',
 'Chemnitz',
 'Saxony',
 'Kiel',
 'Schleswig-Holstein',
 'Aachen',
 'North Rhine-Westphalia',

In [54]:
del districtList[1::2]
districtList

['Berlin',
 'Hamburg',
 'Munich',
 'Cologne',
 'Frankfurt',
 'Stuttgart',
 'Düsseldorf',
 'Dortmund',
 'Essen',
 'Leipzig',
 'Bremen (state)',
 'Dresden',
 'Hanover',
 'Nuremberg',
 'Duisburg',
 'Bochum',
 'Wuppertal',
 'Bielefeld',
 'Bonn',
 'Münster',
 'Karlsruhe',
 'Mannheim',
 'Augsburg',
 'Wiesbaden',
 'Gelsenkirchen',
 'Mönchengladbach',
 'Braunschweig',
 'Chemnitz',
 'Kiel',
 'Aachen',
 'Halle (Saale)',
 'Magdeburg',
 'Freiburg im Breisgau',
 'Krefeld',
 'Lübeck',
 'Oberhausen',
 'Erfurt',
 'Mainz',
 'Rostock',
 'Kassel',
 'Hagen',
 'Hamm',
 'Saarbrücken',
 'Mülheim an der Ruhr',
 'Potsdam',
 'Ludwigshafen am Rhein',
 'Oldenburg (city)',
 'Leverkusen',
 'Osnabrück',
 'Solingen',
 'Heidelberg',
 'Herne, North Rhine-Westphalia',
 'Neuss',
 'Darmstadt',
 'Paderborn',
 'Regensburg',
 'Ingolstadt',
 'Würzburg',
 'Fürth',
 'Wolfsburg',
 'Offenbach am Main',
 'Ulm',
 'Heilbronn',
 'Pforzheim',
 'Göttingen',
 'Bottrop',
 'Trier',
 'Recklinghausen',
 'Reutlingen',
 'Bremerhaven',
 'Koble

In [55]:
# create a new DataFrame from the list
gd_df = pd.DataFrame({"District": districtList})

gd_df

,District
0,Berlin
1,Hamburg
2,Munich
3,Cologne
4,Frankfurt
5,Stuttgart
6,Düsseldorf
7,Dortmund
8,Essen
9,Leipzig


In [56]:
# print the number of rows of the dataframe
gd_df.shape

(79, 1)

##### 3. Get the geographical coordinates

In [57]:
# define a function to get coordinates
def get_latlng(district):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Germany'.format(district))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [59]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(districts) for districts in gd_df["District"].tolist() ]

In [60]:
coords

[[52.51604000000003, 13.376910000000066],
 [53.553750000000036, 9.99183000000005],
 [48.13641000000007, 11.577540000000056],
 [50.941670000000045, 6.955160000000035],
 [50.11208000000005, 8.683410000000038],
 [48.76778000000007, 9.172030000000063],
 [51.215640000000064, 6.776660000000049],
 [51.51661000000007, 7.458290000000034],
 [51.45183000000003, 7.0110800000000495],
 [51.345200000000034, 12.385940000000062],
 [53.06992358200006, 8.736575000000073],
 [51.053630000000055, 13.740810000000067],
 [52.37227000000007, 9.738150000000076],
 [49.45434000000006, 11.07349000000005],
 [51.43148000000008, 6.763550000000066],
 [51.48644000000007, 7.210790000000031],
 [51.271600000000035, 7.196680000000072],
 [52.01547000000005, 8.532310000000052],
 [50.73242000000005, 7.101860000000045],
 [51.963010000000054, 7.617810000000077],
 [49.01094000000006, 8.408450000000073],
 [49.48651000000007, 8.466780000000028],
 [48.370340000000056, 10.897880000000043],
 [50.08466000000004, 8.242140000000063],
 [5

In [61]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [62]:
# merge the coordinates into the original dataframe
gd_df['Latitude'] = df_coords['Latitude']
gd_df['Longitude'] = df_coords['Longitude']

In [63]:
# check the districts and the coordinates
print(gd_df.shape)
gd_df

(79, 3)


,District,Latitude,Longitude
0,Berlin,52.516040,13.376910
1,Hamburg,53.553750,9.991830
2,Munich,48.136410,11.577540
3,Cologne,50.941670,6.955160
4,Frankfurt,50.112080,8.683410
5,Stuttgart,48.767780,9.172030
6,Düsseldorf,51.215640,6.776660
7,Dortmund,51.516610,7.458290
8,Essen,51.451830,7.011080
9,Leipzig,51.345200,12.385940


In [64]:
# save the DataFrame as CSV file
gd_df.to_csv("gd_df.csv", index=False)

##### 4. Create a map of Germany with districts superimposed on top

In [65]:
# get the coordinates of Germany
address = 'Germany'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Germany {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Germany 51.0834196, 10.4234469.


In [66]:
# create map of Germany using latitude and longitude values
map_gd = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, districts in zip(gd_df['Latitude'], gd_df['Longitude'], gd_df['District']):
    label = '{}'.format(districts)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_gd)  
    
map_gd

In [67]:
# save the map as HTML file
map_gd.save('map_gd.html')

##### 5. Use the Foursquare API to explore the districts

In [68]:
# define Foursquare Credentials and Version
CLIENT_ID = 'QZLCWSZMX3IDEBU2SMYXP3J4NX11HXPUFK3BZ3NZ03115AWK' # your Foursquare ID
CLIENT_SECRET = 'VGDX0UG4KDWAQ5CD2QYBCHUBHJ2X4ELNCZ5ROLIOI221PKOL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QZLCWSZMX3IDEBU2SMYXP3J4NX11HXPUFK3BZ3NZ03115AWK
CLIENT_SECRET:VGDX0UG4KDWAQ5CD2QYBCHUBHJ2X4ELNCZ5ROLIOI221PKOL


##### Now, let's get the top 100 venues that are within a radius of 5000 meters.

In [69]:
radius = 10000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(gd_df['Latitude'], gd_df['Longitude'], gd_df['District']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [70]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Districts', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7315, 7)


,Districts,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Berlin,52.51604,13.37691,Brandenburger Tor,52.516247,13.377786,Monument / Landmark
1,Berlin,52.51604,13.37691,Купол Pейхстага (Reichstagskuppel),52.518549,13.376099,Scenic Lookout
2,Berlin,52.51604,13.37691,Reichstag,52.518503,13.376062,Capitol Building
3,Berlin,52.51604,13.37691,Butter Lindner,52.517879,13.380450,Gourmet Shop
4,Berlin,52.51604,13.37691,Dussmann das KulturKaufhaus,52.518343,13.388965,Bookstore


##### Let's check how many venues were returned for each district

In [71]:
venues_df.groupby(["Districts"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Districts,,,,,,
Aachen,100,100,100,100,100,100
Augsburg,100,100,100,100,100,100
Bergisch Gladbach,100,100,100,100,100,100
Berlin,100,100,100,100,100,100
Bielefeld,100,100,100,100,100,100
Bochum,100,100,100,100,100,100
Bonn,100,100,100,100,100,100
Bottrop,100,100,100,100,100,100
Braunschweig,100,100,100,100,100,100


##### Let's find out how many unique categories can be curated from all the returned venues

In [72]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 346 uniques categories.


In [73]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Monument / Landmark', 'Scenic Lookout', 'Capitol Building',
       'Gourmet Shop', 'Bookstore', 'Concert Hall', 'Plaza',
       'Shopping Mall', 'Park', 'Rental Car Location', 'Cycle Studio',
       'Hotel', 'Coffee Shop', 'Garden', 'Caucasian Restaurant', 'Spa',
       'Cocktail Bar', 'Schnitzel Restaurant', 'Sandwich Place',
       'Falafel Restaurant', 'Brewery', 'Ramen Restaurant', 'Bistro',
       'Bakery', 'Art Gallery', 'Vacation Rental', 'Art Museum',
       'Volleyball Court', 'Ice Cream Shop', 'Beer Bar',
       'Arts & Crafts Store', 'Café', 'Event Space', 'Chocolate Shop',
       'Playground', 'Mediterranean Restaurant',
       'Vegetarian / Vegan Restaurant', 'Gym / Fitness Center',
       "Men's Store", 'Movie Theater', 'Boutique', 'Waterfall',
       'French Restaurant', 'Canal', 'Waterfront',
       'Vietnamese Restaurant', 'Spanish Restaurant', 'Bar',
       'Indie Movie Theater', 'Wine Bar'], dtype=object)

In [74]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

##### 6. Analyze each District

In [75]:
# one hot encoding
gd_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add districts column back to dataframe
gd_onehot['Districts'] = venues_df['Districts'] 

# move districts column to the first column
fixed_columns = [gd_onehot.columns[-1]] + list(gd_onehot.columns[:-1])
gd_onehot = gd_onehot[fixed_columns]

print(gd_onehot.shape)
gd_onehot.head()

(7315, 347)


,Districts,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Service,American Restaurant,Amphitheater,Apple Wine Pub,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Bathing Area,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brasserie,Bratwurst Joint,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cable Car,Café,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,College Gym,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Currywurst Joint,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,Fountain,Franconian Restaurant,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Hill,Historic Site,History Museum,Hockey Arena,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Inn,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Kurdish Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundry Service,Light Rail Station,Lighthouse,Liquor Store,Lottery Retailer,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monastery,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,RV Park,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Sho

##### Next, let's group rows by districts and by taking the mean of the frequency of occurrence of each category

In [32]:
gd_grouped = gd_onehot.groupby(["Districts"]).mean().reset_index()

print(gd_grouped.shape)
gd_grouped

(68, 343)


,Districts,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Service,American Restaurant,Amphitheater,Apple Wine Pub,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Bathing Area,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brasserie,Bratwurst Joint,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cable Car,Café,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,College Gym,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Currywurst Joint,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,Fountain,Franconian Restaurant,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Inn,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Kurdish Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundry Service,Light Rail Station,Lighthouse,Liquor Store,Lottery Retailer,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monastery,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,RV Park,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chai

In [33]:
len(gd_grouped[gd_grouped["Shopping Mall"] > 0])

39

##### Create a new DataFrame for Shopping Mall data only

In [34]:
gd_mall = gd_grouped[["Districts","Shopping Mall"]]

In [35]:
gd_mall.head()

,Districts,Shopping Mall
0,Aachen,0.02
1,Augsburg,0.00
2,Berlin,0.01
3,Bielefeld,0.01
4,Bochum,0.01


##### 7. Cluster Districts

##### Run k-means to cluster the districts in Germany into 3 clusters.

In [36]:
# set number of clusters
kclusters = 3

gd_clustering = gd_mall.drop(["Districts"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(gd_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 0, 0, 0, 1, 2, 2, 0, 2], dtype=int32)

In [37]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each district.
gd_merged = gd_mall.copy()

# add clustering labels
gd_merged["Cluster Labels"] = kmeans.labels_

In [38]:
gd_merged.rename(columns={"Districts": "District"}, inplace=True)
gd_merged.head()

,District,Shopping Mall,Cluster Labels
0,Aachen,0.02,2
1,Augsburg,0.00,1
2,Berlin,0.01,0
3,Bielefeld,0.01,0
4,Bochum,0.01,0


In [41]:
# merge Germany_grouped with germany_data to add latitude/longitude for each neighborhood
gd_merged = gd_merged.join(gd_df.set_index("District"), on="District")

print(gd_merged.shape)
gd_merged.head() # check the last columns!

(68, 5)


,District,Shopping Mall,Cluster Labels,Latitude,Longitude
17,Essen,0.010000,0,51.45183,7.01108
22,Göttingen,0.014493,0,51.53119,9.93805
25,Hamburg,0.010000,0,53.55375,9.99183
57,Regensburg,0.010204,0,49.01492,12.10173
58,Rostock,0.011111,0,54.07901,12.13231


In [42]:
# sort the results by Cluster Labels
print(gd_merged.shape)
gd_merged.sort_values(["Cluster Labels"], inplace=True)
gd_merged

(68, 5)


,District,Shopping Mall,Cluster Labels,Latitude,Longitude
17,Essen,0.010000,0,51.451830,7.011080
24,Halle (Saale),0.010000,0,51.482200,11.974940
11,Darmstadt,0.010000,0,49.872640,8.650130
32,Karlsruhe,0.010000,0,49.010940,8.408450
2,Berlin,0.010000,0,52.516040,13.376910
3,Bielefeld,0.010000,0,52.015470,8.532310
51,Oldenburg (city),0.013514,0,53.138820,8.211240
31,Ingolstadt,0.010638,0,48.762360,11.425230
30,"Herne, North Rhine-Westphalia",0.010000,0,51.540310,7.219840
8,Bremen (state),0.010000,0,53.069924,8.736575


##### Finally, let's visualize the resulting clusters

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(gd_merged['Latitude'], gd_merged['Longitude'], gd_merged['District'], gd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

##### 8. Examine Clusters

##### Cluster 0

In [45]:
gd_merged.loc[gd_merged['Cluster Labels'] == 0]

,District,Shopping Mall,Cluster Labels,Latitude,Longitude
17,Essen,0.010000,0,51.451830,7.011080
24,Halle (Saale),0.010000,0,51.482200,11.974940
11,Darmstadt,0.010000,0,49.872640,8.650130
32,Karlsruhe,0.010000,0,49.010940,8.408450
2,Berlin,0.010000,0,52.516040,13.376910
3,Bielefeld,0.010000,0,52.015470,8.532310
51,Oldenburg (city),0.013514,0,53.138820,8.211240
31,Ingolstadt,0.010638,0,48.762360,11.425230
30,"Herne, North Rhine-Westphalia",0.010000,0,51.540310,7.219840
8,Bremen (state),0.010000,0,53.069924,8.736575


##### Cluster 1

In [46]:
gd_merged.loc[gd_merged['Cluster Labels'] == 1]

,District,Shopping Mall,Cluster Labels,Latitude,Longitude
20,Fürth,0.0,1,49.47317,10.99065
61,Stuttgart,0.0,1,48.76778,9.17203
15,Düsseldorf,0.0,1,51.21564,6.77666
19,Freiburg im Breisgau,0.0,1,47.99853,7.84965
14,Duisburg,0.0,1,51.43148,6.76355
21,Gelsenkirchen,0.0,1,51.51152,7.09308
54,Pforzheim,0.0,1,48.89011,8.70013
26,Hamm,0.0,1,51.68010,7.81752
53,Paderborn,0.0,1,51.71814,8.75204
28,Heidelberg,0.0,1,49.41349,8.70798


##### Cluster 2

In [47]:
gd_merged.loc[gd_merged['Cluster Labels'] == 2]

,District,Shopping Mall,Cluster Labels,Latitude,Longitude
6,Bottrop,0.030000,2,51.52470,6.92520
7,Braunschweig,0.020000,2,52.26544,10.52740
9,Chemnitz,0.023810,2,50.83669,12.92087
23,Hagen,0.033898,2,51.35990,7.47129
34,Kiel,0.020000,2,54.32325,10.13224
36,Leipzig,0.020000,2,51.34520,12.38594
42,Mannheim,0.020000,2,49.48651,8.46678
39,Lübeck,0.043478,2,53.86627,10.68082
46,Münster,0.020000,2,51.96301,7.61781
0,Aachen,0.020000,2,50.77823,6.08864


#### Summary - Observations